In [ ]:
import numpy as np
import pandas as pd
!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
import pandas as pd

files = [
    'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv',
    '1429_1.csv'
]
selected_columns = ['text', 'rating']


dfs = [pd.read_csv(file, usecols=selected_columns) for file in files]

df = pd.concat(dfs, ignore_index=True)

df.head()

df.info()


In [ ]:
df.isnull().sum()

In [ ]:
# df['rating'] = df['rating'].fillna(df['rating'].median())df.dropna()

In [ ]:
df_clean=df.dropna()

In [ ]:
df_clean.isnull().sum()

In [ ]:
def label_sentiment(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

df['sentiment'] = df['rating'].apply(label_sentiment)
df_clean.info()

In [ ]:
import re
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# -----------------------------
# Load spaCy model efficiently
# -----------------------------
# Disable parser and NER for speed, only use tokenizer & tagger
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# -----------------------------
# Precompile regex for cleaning
# -----------------------------
RE_CLEAN = re.compile(r'[^a-zA-Z\s]')  # keep only letters and spaces

# -----------------------------
# Function to clean a single doc
# -----------------------------
def clean_doc(doc):
    tokens = [
        token.lemma_  # lemmatize
        for token in doc
        if token.is_alpha and token.text not in STOP_WORDS
    ]
    return " ".join(tokens)

# -----------------------------
# Prepare text column
# -----------------------------
texts = df_clean['text'].astype(str).str.lower().tolist()  # lowercase all

# -----------------------------
# Batch process using spaCy pipe
# -----------------------------
# Adjust batch_size and n_process based on dataset size and CPU cores
cleaned_texts = [
    clean_doc(doc)
    for doc in nlp.pipe(texts, batch_size=1000, n_process=2)
]

# -----------------------------
# Save cleaned text back to DataFrame
# -----------------------------
df_clean['clean_text'] = cleaned_texts

# -----------------------------
# Quick check
# -----------------------------
print(df_clean[['text', 'clean_text']].head())
